# 2 start pods on minikube

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/roboticbase-core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/roboticbase-core
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/minikube/env

## start RabbitMQ cluster on minikube
[rabbitmq](https://www.rabbitmq.com/)

### start RabbitMQ

In [ ]:
kubectl apply -f rabbitmq/rabbitmq-rbac.yaml

In [ ]:
kubectl apply -f rabbitmq/rabbitmq-minikube-services.yaml

In [ ]:
kubectl apply -f rabbitmq/rabbitmq-minikube-statefulset.yaml

In [ ]:
kubectl get pods -l app=rabbitmq

example)

```
NAME         READY     STATUS    RESTARTS   AGE
rabbitmq-0   1/1       Running   0          6m
rabbitmq-1   1/1       Running   0          5m
rabbitmq-2   1/1       Running   0          4m
```

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl cluster_status

example)
```
Cluster status of node rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local ...
[{nodes,[{disc,['rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local',
                'rabbit@rabbitmq-1.rabbitmq.default.svc.cluster.local',
                'rabbit@rabbitmq-2.rabbitmq.default.svc.cluster.local']}]},
 {running_nodes,['rabbit@rabbitmq-2.rabbitmq.default.svc.cluster.local',
                 'rabbit@rabbitmq-1.rabbitmq.default.svc.cluster.local',
                 'rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local']},
 {cluster_name,<<"rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local">>},
 {partitions,[]},
 {alarms,[{'rabbit@rabbitmq-2.rabbitmq.default.svc.cluster.local',[]},
          {'rabbit@rabbitmq-1.rabbitmq.default.svc.cluster.local',[]},
          {'rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local',[]}]}]
```

### change password of `guest`

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl change_password guest $(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32)

### register users to RabbitMQ

In [ ]:
for e in $(env); do
  if [[ "${e}" =~ ^MQTT__([[:alnum:]_-]+)=([[:alnum:]_-]+)$ ]]; then
    username=${BASH_REMATCH[1]}
    password=${BASH_REMATCH[2]}
    
    kubectl exec rabbitmq-0 -- rabbitmqctl add_user ${username} ${password}
    kubectl exec rabbitmq-0 -- rabbitmqctl set_permissions -p / ${username} ".*" ".*" ".*"
  fi
done

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl list_users

example)
```
Listing users ...
ros	[]
raspberrypi	[]
guest	[administrator]
iotagent	[]
```

In [ ]:
kubectl get services -l app=rabbitmq

expected)
```
NAME            TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)              AGE
rabbitmq        ClusterIP   None             <none>        5672/TCP             4m
rabbitmq-amqp   ClusterIP   10.107.253.173   <none>        15672/TCP,5672/TCP   4m
rabbitmq-mqtt   NodePort    10.102.140.44    <none>        1883:31186/TCP       4m
```

### nat MQTT port 

In [ ]:
export MQTT_PORT=$(kubectl describe service rabbitmq-mqtt | grep "NodePort:" | awk '{print $3}' | awk -F'/' '{print $1}');echo ${MQTT_PORT}

In [ ]:
VBoxManage controlvm "minikube" natpf1 "mqtt,tcp,0.0.0.0,1883,,${MQTT_PORT}"
VBoxManage showvminfo "minikube" | grep ${MQTT_PORT}

example)
```
NIC 1 Rule(0):   name = mqtt, protocol = tcp, host ip = 0.0.0.0, host port = 1883, guest ip = , guest port = 30382
```

### confirm to connect MQTT broker

In [ ]:
mosquitto_pub -h ${HOST_IPADDR} -p 1883  -d -u iotagent -P ${MQTT__iotagent} -t /test -m "test"

example)
```
Client mosqpub|52150-Nobuyukin sending CONNECT
Client mosqpub|52150-Nobuyukin received CONNACK
Client mosqpub|52150-Nobuyukin sending PUBLISH (d0, q0, r0, m1, '/test', ... (4 bytes))
Client mosqpub|52150-Nobuyukin sending DISCONNECT
```

## start mondodb cluster on minikube
[mongodb](https://www.mongodb.com/)

In [ ]:
kubectl apply -f mongodb/mongodb-cluster-minikube.yaml

In [ ]:
kubectl get pods -l app=mongodb

example)
```
NAME        READY     STATUS    RESTARTS   AGE
mongodb-0   2/2       Running   0          6m
mongodb-1   2/2       Running   0          4m
mongodb-2   2/2       Running   0          2m
```

In [ ]:
kubectl get services -l app=mongodb

example)
```
NAME      TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)     AGE
mongodb   ClusterIP   None         <none>        27017/TCP   6m
```

In [ ]:
kubectl exec mongodb-0 -c mongodb -- mongo --eval 'printjson(rs.status().members.map(function(e) {return {name: e.name, stateStr:e.stateStr};}))'

expected)
```
MongoDB shell version v3.6.6
connecting to: mongodb://127.0.0.1:27017
MongoDB server version: 3.6.6
[
	{
		"name" : "mongodb-0.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "PRIMARY"
	},
	{
		"name" : "mongodb-1.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "SECONDARY"
	},
	{
		"name" : "mongodb-2.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "SECONDARY"
	}
]
```

## start ambassador on minikube

[ambassador](https://www.getambassador.io/)

### start ambassador

In [ ]:
kubectl apply -f ambassador/ambassador-minikube-service.yaml

In [ ]:
kubectl apply -f ambassador/ambassador-deployment.yaml

In [ ]:
kubectl get pods -l service=ambassador

example)
```
NAME                         READY     STATUS    RESTARTS   AGE
ambassador-cf99ff6bb-45jpl   1/1       Running   0          39s
ambassador-cf99ff6bb-8kht7   1/1       Running   0          39s
ambassador-cf99ff6bb-lv2tk   1/1       Running   0          39s
```

In [ ]:
kubectl get services -l service=ambassador

example)
```
NAME         TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
ambassador   NodePort   10.98.181.146   <none>        80:31807/TCP   1m
```

### nat HTTP port

In [ ]:
HTTP_PORT=$(kubectl describe service ambassador | grep "NodePort:" | awk '{print $3}' | awk -F'/' '{print $1}');echo ${HTTP_PORT}

In [ ]:
VBoxManage controlvm "minikube" natpf1 "http,tcp,0.0.0.0,8080,,${HTTP_PORT}"
VBoxManage showvminfo "minikube" | grep ${HTTP_PORT}

In [ ]:
curl -i http://${HOST_IPADDR}:8080

expected)
```
HTTP/1.1 404 Not Found
date: Fri, 25 May 2018 00:47:41 GMT
server: envoy
content-length: 0
```

## start authorization & authentication service on minikube

### create `secrets/auth-tokens.json`
example)
```json
{
  "bearer_tokens": [
      {
          "token": "iRGTsKKHwgjf4rR2XMSN3oE9Dhm6ym3O",
          "allowed_paths": ["^/orion/.*$", "^/idas/.*$"]
      }, {
          "token": "4Xc1GFa2D8zkZRbkdygm902oGYeUAJno",
          "allowed_paths": ["^/visualizer/positions/$"]
      }
  ],
  "basic_auths": [
      {
          "username": "user1",
          "password": "P@ssw0rd",
          "allowed_paths": ["/controller/web/"]
      }, {
          "username": "visualizer",
          "password": "P@ssw0rd",
          "allowed_paths": ["/visualizer/locus/"]
      }
  ]
}
```

In [ ]:
cat << __EOS__ > secrets/auth-tokens.json
{
  "bearer_tokens": [
      {
          "token": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32)",
          "allowed_paths": ["^/orion/.*$", "^/idas/.*$"]
      }, {
          "token": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32)",
          "allowed_paths": ["^/visualizer/positions/$"]
      }
  ],
  "basic_auths": [
      {
          "username": "user1",
          "password": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 16)",
          "allowed_paths": ["/controller/web/"]
      }, {
          "username": "visualizer",
          "password": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 16)",
          "allowed_paths": ["/visualizer/locus/"]
      }
  ]
}
__EOS__

### register auth-tokens to kubernetes secrets

In [ ]:
kubectl create secret generic auth-tokens --from-file=./secrets/auth-tokens.json

### start fiware-ambassador-auth

In [ ]:
kubectl apply -f ambassador/fiware-ambassador-auth.yaml

In [ ]:
kubectl get pods -l pod=ambassador-auth

example)
```
NAME                           READY     STATUS    RESTARTS   AGE
ambassador-auth-6fffdbd9c9-7kkpr   1/1       Running   0          56s
ambassador-auth-6fffdbd9c9-qxw6m   1/1       Running   0          56s
ambassador-auth-6fffdbd9c9-sdn5b   1/1       Running   0          56s
```

In [ ]:
kubectl get services -l service=ambassador-auth

example)
```
NAME          TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
ambassador-auth   ClusterIP   10.0.129.102   <none>        3000/TCP   2m
```

In [ ]:
curl -i http://${HOST_IPADDR}:8080

example)
```
HTTP/1.1 401 Unauthorized
content-type: application/json; charset=utf-8
www-authenticate: Bearer realm="token_required"
date: Thu, 12 Jul 2018 02:21:24 GMT
content-length: 60
x-envoy-upstream-service-time: 2
server: envoy

{"authorized":false,"error":"missing Header: authorization"}
```

## start fiware orion on minikube
[fiware orion](https://catalogue-server.fiware.org/enablers/publishsubscribe-context-broker-orion-context-broker)

In [ ]:
kubectl apply -f orion/orion.yaml

In [ ]:
kubectl get pods -l app=orion

example)
```
NAME                     READY     STATUS    RESTARTS   AGE
orion-54f5cdcb5d-d2pt5   1/1       Running   0          56s
orion-54f5cdcb5d-hv274   1/1       Running   0          56s
orion-54f5cdcb5d-xbnx2   1/1       Running   0          56s
```

In [ ]:
kubectl get services -l app=orion

example)
```
NAME      TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
orion     ClusterIP   10.0.44.126   <none>        1026/TCP   1m
```

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" http://${HOST_IPADDR}:8080/orion/v2/entities/

example)
```
HTTP/1.1 200 OK
content-length: 2
content-type: application/json
fiware-correlator: 4731eb48-4dc1-11e8-b1a2-0a580af4010a
date: Wed, 02 May 2018 04:28:35 GMT
x-envoy-upstream-service-time: 5
server: envoy

[]
```
**If you can't get 200 OK, please restart all ambassador's pods like "ambassador-cf99ff6bb-45jpl", "ambassador-cf99ff6bb-8kht7" and "ambassador-cf99ff6bb-lv2tk"**

## start fiware idas(iotagent-ul) on minikube
[fiware IDAS(iotagent-ul)](https://catalogue-server.fiware.org/enablers/backend-device-management-idas)

**In this demonstration, we use customized iotagent-ul in order to ignore duplicate MQTT messages.**

### build and push the idas conatiner image

In [ ]:
docker build -t ${REPOSITORY}/tech-sketch/iotagent-ul:290a1fa idas/iotagent-ul/

In [ ]:
docker push ${REPOSITORY}/tech-sketch/iotagent-ul:290a1fa

expected)
```
The push refers to repository [192.168.99.1:5000/tech-sketch/iotagent-ul]

1f141107: Preparing 
e3e1d3a9: Preparing 
a102cc1f: Preparing 
290a1fa: digest: sha256:468a6c2e9aaf6d47a4e21d95bbdb9f8c2df088e876e0846ceaba09cca5fc2d13 size: 1163
```

In [ ]:
docker image ls | grep iotagent-ul | grep 290a1fa

example)
```
192.168.99.1:5000/tech-sketch/iotagent-ul                         290a1fa                992c1909ec08        23 hours ago        112MB
```

### start idas

In [ ]:
env IOTA_PASSWORD=${MQTT__iotagent} envsubst < idas/config.js > /tmp/config.js
kubectl create secret generic iotagent-config --from-file /tmp/config.js
rm /tmp/config.js

In [ ]:
envsubst < idas/iotagent-ul.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l app=iotagent-ul

example)
```
NAME                           READY     STATUS    RESTARTS   AGE
iotagent-ul-79685b64bf-8krps   1/1       Running   0          3m
iotagent-ul-79685b64bf-m6nlg   1/1       Running   0          3m
iotagent-ul-79685b64bf-mjpbl   1/1       Running   0          3m
```

In [ ]:
kubectl get services -l app=iotagent-ul

example)
```
NAME          TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)             AGE
iotagent-ul   ClusterIP   10.0.180.155   <none>        4041/TCP,7896/TCP   43s
```

In [ ]:
TOKEN=$(cat secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: /*" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/services/

example)
```
HTTP/1.1 200 OK
x-powered-by: Express
fiware-correlator: c114fc5e-b4a2-40f6-b7fe-1d68369784e5
content-type: application/json; charset=utf-8
content-length: 25
etag: W/"19-WMYe0U6ocKhQjp+oaVnMHLdbylc"
date: Wed, 02 May 2018 06:16:18 GMT
x-envoy-upstream-service-time: 9
server: envoy

{"count":0,"services":[]}
```

## start fiware cygnus on minikube
[fiware cygnus](https://catalogue-server.fiware.org/enablers/cygnus)

### build and push the cygnus conatiner image

In [ ]:
docker build -t ${REPOSITORY}/tech-sketch/cygnus-ngsi:1.9.0 ./cygnus/fiware-cygnus/

In [ ]:
docker push ${REPOSITORY}/tech-sketch/cygnus-ngsi:1.9.0

expected)
```
The push refers to repository [192.168.99.1:5000/tech-sketch/cygnus-ngsi]

d0cfd3c5: Preparing 
dc1ea792: Preparing 
4942fce2: Preparing 
1a59431b: Preparing 
3593b4b9: Preparing 
95b7aae7: Preparing 
6c9f6afa: Preparing 
e5b8d4c2: Preparing 
1.9.0: digest: sha256:f1a8f553fe983fa980687150af885b56fb423aceef4050137a2ad20a2bd63a83 size: 2198
```

In [ ]:
docker image ls | grep cygnus-ngsi | grep 1.9.0

example)
```
192.168.99.1:5000/tech-sketch/cygnus-ngsi                         1.9.0                  20238d7d0b15        9 days ago          516MB
```

### start cygnus

In [ ]:
envsubst < cygnus/cygnus.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l app=cygnus

example)
```
NAME                      READY     STATUS    RESTARTS   AGE
cygnus-5c68fb6578-fdmtg   1/1       Running   0          44s
cygnus-5c68fb6578-stmds   1/1       Running   0          44s
cygnus-5c68fb6578-z85lp   1/1       Running   0          44s
```

In [ ]:
kubectl get services -l app=cygnus

example)
```
NAME      TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)             AGE
cygnus    ClusterIP   10.103.255.240   <none>        5050/TCP,8081/TCP   1m
```

## start command proxy service on minikube

In [ ]:
envsubst < controller/fiware-cmd-proxy.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l pod=cmd-proxy

example)
```
NAME                        READY     STATUS    RESTARTS   AGE
cmd-proxy-d7cbc6dfb-6hnsp   1/1       Running   0          46s
cmd-proxy-d7cbc6dfb-v6rrx   1/1       Running   0          46s
cmd-proxy-d7cbc6dfb-zf2fd   1/1       Running   0          46s
```

In [ ]:
kubectl get services -l service=cmd-proxy

example)
```
NAME        TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
cmd-proxy   ClusterIP   10.0.172.56   <none>        8888/TCP   1m
```

## start robot visualization service on minikube

In [ ]:
export MONGODB_DATABASE="sth_${FIWARE_SERVICE}"
export MONGODB_COLLECTION="sth_${ROBOT_SERVICEPATH}_${ROBOT_ID}_${ROBOT_TYPE}"
env BEARER_AUTH=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens | map(select(.allowed_paths[] | contains ("^/visualizer/positions/$"))) | .[0].token' -r) envsubst < controller/fiware-robot-visualization.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l pod=robot-visualization

example)
```
NAME                                   READY     STATUS    RESTARTS   AGE
robot-visualization-7d69bcdff7-g7p98   1/1       Running   0          13m
robot-visualization-7d69bcdff7-k6mb9   1/1       Running   0          13m
robot-visualization-7d69bcdff7-txsqn   1/1       Running   0          13m
```

In [ ]:
kubectl get services -l service=robot-visualization

example)
```
NAME                  TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
robot-visualization   ClusterIP   10.0.112.72   <none>        8888/TCP   13m
```